In [81]:
import os

In [82]:
pwd

'c:\\Users\\abbaig\\Krish naik course\\heart-disease-prediction'

In [83]:
os.chdir('c:\\Users\\abbaig\\Krish naik course\\heart-disease-prediction')

In [88]:
## Update entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL : str
    local_data_file: Path
    unzip_dir : Path

In [89]:
## update the configuration manager
from heart_disease_project.constants import *
from heart_disease_project.utils.common import read_yaml,create_directories

    

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            local_data_file=config.local_data_file,
            source_URL=config.source_URL,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [90]:
## update the components

import os 
import urllib.request as request 
import zipfile
from heart_disease_project import logger
from heart_disease_project.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [91]:
## update the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-24 15:04:16,454: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-24 15:04:16,454: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-24 15:04:16,462: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-24 15:04:16,462: INFO: common: created directory at: artifacts]
[2024-03-24 15:04:16,468: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-24 15:04:17,470: INFO: 3616834972: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 1553
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d5a7134c9e3b7e4138730b3c75ea9df8bc340b8d1f1d46887c1452465f1e39b0"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7DD2:24BF3D:186396:215446:65FFF398
Accept-Ranges: bytes
Date: Sun, 24 